### CCFD 2.0

In [1]:
import numpy as np
import pandas as pd
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('CCFD New Dataset.csv')
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",3/9/1988,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,6/21/1978,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1/19/1962,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1/12/1967,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,3/28/1986,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [3]:
now = pd.Timestamp('now')
data['dob'] = pd.to_datetime(data['dob'])    # 1
data['dob'] = data['dob'].where(data['dob'] < now, data['dob'] -  np.timedelta64(100, 'Y'))   # 2
data['age'] = (now - data['dob']).astype('<m8[Y]')    # 3
print(data)

         Unnamed: 0 trans_date_trans_time        cc_num  \
0                 0         1/1/2019 0:00  2.703190e+15   
1                 1         1/1/2019 0:00  6.304230e+11   
2                 2         1/1/2019 0:00  3.885950e+13   
3                 3         1/1/2019 0:01  3.534090e+15   
4                 4         1/1/2019 0:03  3.755340e+14   
...             ...                   ...           ...   
1048570     1048570       3/10/2020 16:07  6.011980e+15   
1048571     1048571       3/10/2020 16:07  4.839040e+15   
1048572     1048572       3/10/2020 16:08  5.718440e+11   
1048573     1048573       3/10/2020 16:08  4.646850e+18   
1048574     1048574       3/10/2020 16:08  2.283740e+15   

                                   merchant        category     amt  \
0                fraud_Rippin, Kub and Mann        misc_net    4.97   
1           fraud_Heller, Gutmann and Zieme     grocery_pos  107.23   
2                      fraud_Lind-Buckridge   entertainment  220.11   
3      

In [4]:
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,33.0
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,43.0
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,59.0
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,54.0
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,35.0


In [5]:
data ["sex"] = 0
data.loc[data["gender"]== "F", "sex"]= 1  
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,sex
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,33.0,1
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,43.0,1
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,59.0,0
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,54.0,0
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,35.0,0


In [6]:
data['Date']=pd.to_datetime(data['trans_date_trans_time']).dt.date
data['Time']=pd.to_datetime(data['trans_date_trans_time']).dt.time
data['Time']=data['Time'].astype(str)
data['Time']=data['Time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) + int(x[2])/60)
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,age,sex,Date,Time
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,33.0,1,2019-01-01,0.0
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,43.0,1,2019-01-01,0.0
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,59.0,0,2019-01-01,0.0
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,54.0,0,2019-01-01,1.0
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,35.0,0,2019-01-01,3.0


In [7]:
data['Tran_State']=0
data['Tran_Category']=0
data['Cust_City']=0
f=pd.DataFrame(data["state"])
e={}
z=0
for i in f.iterrows():
    if i[1]["state"] in e.keys():
        i[1]["state"]=e[i[1]["state"]]
    else:
        e[i[1]["state"]]=z
        i[1]["state"] = e[i[1]["state"]]
        z+=1

data['Tran_State']=f['state']

c=pd.DataFrame(data["category"])
d={}
y=0
for i in c.iterrows():
    if i[1]["category"] in d.keys():
        i[1]["category"]=d[i[1]["category"]]
    else:
        d[i[1]["category"]]=y
        i[1]["category"] = d[i[1]["category"]]
        y+=1

data['Tran_Category']=c['category']

f=pd.DataFrame(data["city"])
e={}
z=0
for i in f.iterrows():
    if i[1]["city"] in e.keys():
        i[1]["city"]=e[i[1]["city"]]
    else:
        e[i[1]["city"]]=z
        i[1]["city"] = e[i[1]["city"]]
        z+=1
data['Cust_City']=f['city']
data.head()


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merch_lat,merch_long,is_fraud,age,sex,Date,Time,Tran_State,Tran_Category,Cust_City
0,0,1/1/2019 0:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.011293,-82.048315,0,33.0,1,2019-01-01,0.0,0,0,0
1,1,1/1/2019 0:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,49.159047,-118.186462,0,43.0,1,2019-01-01,0.0,1,1,1
2,2,1/1/2019 0:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,43.150704,-112.154481,0,59.0,0,2019-01-01,0.0,2,2,2
3,3,1/1/2019 0:01,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,47.034331,-112.561071,0,54.0,0,2019-01-01,1.0,3,3,3
4,4,1/1/2019 0:03,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.674999,-78.632459,0,35.0,0,2019-01-01,3.0,4,4,4


In [8]:
data[["Tran_State", "Cust_City", "Tran_Category"]] = data[["Tran_State", "Cust_City", "Tran_Category"]].apply(pd.to_numeric)

In [9]:
tt=data.to_csv(r'C:/Users/Md. Kaviul Hossain/Desktop/CCFD_temp.csv', index = False)

In [10]:
file = 'C:/Users/Md. Kaviul Hossain/Desktop/CCFD_temp.csv'
data = pd.read_csv(file, sep = ',', parse_dates= ['Date'],encoding='utf-8-sig', usecols= ['Date', 'Time', 'is_fraud', 'amt', 'zip', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'Tran_Category', 'Cust_City', 'Tran_State', 'age', 'sex'],)    
data['Month'] = data['Date'].dt.month


In [11]:
data.tail()


,amt,zip,lat,long,city_pop,merch_lat,merch_long,is_fraud,age,sex,Date,Time,Tran_State,Tran_Category,Cust_City,Month
1048570,77.00,21405,39.0305,-76.5515,92106,38.779464,-76.317042,0,78.0,1,2020-03-10,967.0,32,10,353,3
1048571,116.94,52563,41.1826,-92.3097,1583,41.400318,-92.726724,0,22.0,1,2020-03-10,967.0,8,4,773,3
1048572,21.27,40202,38.2507,-85.7476,736284,37.293339,-84.798122,0,69.0,1,2020-03-10,968.0,26,13,245,3
1048573,9.52,11796,40.7320,-73.1000,4056,39.773077,-72.213209,0,31.0,1,2020-03-10,968.0,27,10,143,3
1048574,6.81,30009,34.0770,-84.3033,165556,33.601468,-83.891921,0,23.0,1,2020-03-10,968.0,33,4,548,3


In [12]:
data=data[['Time','amt','zip','lat','long','city_pop','merch_lat','merch_long', 'Cust_City', 'Tran_State', 'Tran_Category', 'Month','age','sex','is_fraud']]

In [13]:
data.head()

,Time,amt,zip,lat,long,city_pop,merch_lat,merch_long,Cust_City,Tran_State,Tran_Category,Month,age,sex,is_fraud
0,0.0,4.97,28654,36.0788,-81.1781,3495,36.011293,-82.048315,0,0,0,1,33.0,1,0
1,0.0,107.23,99160,48.8878,-118.2105,149,49.159047,-118.186462,1,1,1,1,43.0,1,0
2,0.0,220.11,83252,42.1808,-112.2620,4154,43.150704,-112.154481,2,2,2,1,59.0,0,0
3,1.0,45.00,59632,46.2306,-112.1138,1939,47.034331,-112.561071,3,3,3,1,54.0,0,0
4,3.0,41.96,24433,38.4207,-79.4629,99,38.674999,-78.632459,4,4,4,1,35.0,0,0


In [14]:
tt1=data.to_csv(r'C:/Users/Md. Kaviul Hossain/Desktop/CCFD_Final_Dataset.csv', index = False)